In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [37]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [38]:
# import data
data_dir = 'data/hymenoptera_data'
sets = ['train', 'val']
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                         data_transforms[x])
                 for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

data_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

['ants', 'bees']


In [39]:
data_sizes

{'train': 244, 'val': 153}

In [40]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 244
     Root location: data/hymenoptera_data\train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 153
     Root location: data/hymenoptera_data\val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
            )}

In [41]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # set model to training mode
            else:
                model.eval() # set model to evaluate mode
            
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize if only in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_corrects.double() / data_sizes[phase]
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
          
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
    

In [42]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

In [43]:
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [45]:
# scheduler - this updates the learning rate
# every 7 (step_size) epochs our learning rate gets multiplied by 0.1 (gamma)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [46]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.6159 Acc: 0.6393
val Loss: 0.4567 Acc: 0.7974

Epoch 1/19
----------
train Loss: 0.5703 Acc: 0.6967
val Loss: 0.3629 Acc: 0.8954

Epoch 2/19
----------
train Loss: 0.5046 Acc: 0.7910
val Loss: 0.2985 Acc: 0.9150

Epoch 3/19
----------
train Loss: 0.4239 Acc: 0.8279
val Loss: 0.2728 Acc: 0.9281

Epoch 4/19
----------
train Loss: 0.4749 Acc: 0.7869
val Loss: 0.2366 Acc: 0.9412

Epoch 5/19
----------
train Loss: 0.3907 Acc: 0.8115
val Loss: 0.2242 Acc: 0.9346

Epoch 6/19
----------
train Loss: 0.3936 Acc: 0.8074
val Loss: 0.1986 Acc: 0.9412

Epoch 7/19
----------
train Loss: 0.3697 Acc: 0.8484
val Loss: 0.2026 Acc: 0.9346

Epoch 8/19
----------
train Loss: 0.4074 Acc: 0.8074
val Loss: 0.2035 Acc: 0.9412

Epoch 9/19
----------
train Loss: 0.3322 Acc: 0.8566
val Loss: 0.2009 Acc: 0.9346

Epoch 10/19
----------
train Loss: 0.3745 Acc: 0.8320
val Loss: 0.1918 Acc: 0.9281

Epoch 11/19
----------
train Loss: 0.3328 Acc: 0.8770
val Loss: 0.2012 Acc: 0.9412

Ep

In [47]:
# the above method is called fine-tuning
# we can also freeze all the layers in the beginning and only train the last layer that we modified
model = models.resnet18(pretrained=True)
for param in model.parameters():
    # this freezes the layers in the beginning
    param.requires_grad = False

In [48]:
num_ftrs = model.fc.in_features

In [49]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [50]:
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [51]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.8117 Acc: 0.5041
val Loss: 0.8180 Acc: 0.5490

Epoch 1/19
----------
train Loss: 0.8074 Acc: 0.5082
val Loss: 0.8181 Acc: 0.5294

Epoch 2/19
----------
train Loss: 0.8093 Acc: 0.5041
val Loss: 0.7899 Acc: 0.5359

Epoch 3/19
----------
train Loss: 0.8051 Acc: 0.5041
val Loss: 0.7979 Acc: 0.5425

Epoch 4/19
----------
train Loss: 0.8234 Acc: 0.5041
val Loss: 0.8665 Acc: 0.5294

Epoch 5/19
----------
train Loss: 0.8046 Acc: 0.5041
val Loss: 0.8344 Acc: 0.5229

Epoch 6/19
----------
train Loss: 0.8380 Acc: 0.4959
val Loss: 0.8238 Acc: 0.5229

Epoch 7/19
----------
train Loss: 0.8443 Acc: 0.5082
val Loss: 0.8145 Acc: 0.5294

Epoch 8/19
----------
train Loss: 0.8298 Acc: 0.5082
val Loss: 0.8367 Acc: 0.5294

Epoch 9/19
----------
train Loss: 0.8232 Acc: 0.5123
val Loss: 0.8323 Acc: 0.5359

Epoch 10/19
----------
train Loss: 0.8087 Acc: 0.5082
val Loss: 0.8371 Acc: 0.5229

Epoch 11/19
----------
train Loss: 0.8029 Acc: 0.5041
val Loss: 0.8252 Acc: 0.5229

Ep